In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

2024-12-31 19:15:26.781631: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
df = pd.read_csv("./Sample_data/winemag-data-130k-v2.csv", usecols = ['country','description', 'points', 'price','variety','winery'])

In [ ]:
df.head()

In [ ]:
df = df.dropna(subset=["description", "points"])
df.head(10)

In [ ]:
plt.hist(df.points, bins=20)
plt.title("Points histogram")
plt.ylabel("N")
plt.xlabel("points")
plt.show()

In [ ]:
df["label"] = (df.points >= 90).astype(int)
df = df[["description" , "points" , "label"]]
df.head()

In [ ]:
train,val,test = np.split(df.sample(frac=1), [int(0.8*len(df)),int(0.9*len(df))])
len(train), len(val),len(test)

In [ ]:
def df_to_dataset(dataframe,shuffle=True, batch_size=1024):
    df = dataframe.copy()
    labels = df.pop("label")
    df = df["description"]
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)    
    return ds

In [ ]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)


Embedding and model


In [ ]:
embedding = "https://www.kaggle.com/models/google/nnlm/TensorFlow2/en-dim50/1"
hub_layer = hub.KerasLayer(embedding,dtype=tf.string,trainable=True)
#hub_layer = hub.KerasLayer("https://www.kaggle.com/models/google/nnlm/TensorFlow2/en-dim50/1" ,input_shape=[], dtype=tf.string)

In [ ]:
hub_layer(list(train_data)[0][0])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
model.evaluate(train_data)


In [ ]:
model.evaluate(valid_data)

In [ ]:
history = model.fit(train_data, epochs=10, valid_data=valid_data)

In [ ]:
plt.plot(history.history["acc"], label="validation acc")
plt.plot(history.history["val_acc"], label="validation acc")
plt.title("accuracy of model")
plt.ylabel("Accuracy")
plt.xlabel("epoch")
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_data)

LSTM

In [ ]:
encoder = tf.keras.layers.TextVectorization(max_tokens=2000)
encoder.adapt(train_data.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        mask_zero=True
    ),
    tf.keras.layers.LTSM(32)
    tf.keras.layers.Dense(32,, activation='relu')
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
model.evaluate(train_data)
model.evaluate(valid_data)

In [ ]:
history = model.fit(train_data,epochs=5,validation_data=valid_data)

In [ ]:
model.evaluate(test_data)